# Compulsory assignment 1 - Using Dask-ml on large data
## Group #16
### Thomas Moen and Jørgen Kongsro

In [1]:
%matplotlib notebook


import numpy as np
import pandas as pd
import dask
import os


from operator import itemgetter

import numpy as np
import scipy

import dask.dataframe as dd
from dask.diagnostics import ProgressBar

import matplotlib.pyplot as plt
import skimage.io

from dask_ml.linear_model import LogisticRegression
from dask_ml.model_selection import train_test_split


ModuleNotFoundError: No module named 'dask_ml'

In [ ]:
# Install Kaggle API
# How to setup: https://github.com/Kaggle/kaggle-api
# or visit: https://adityashrm21.github.io/Setting-Up-Kaggle/

#!pip install kaggle

In [ ]:
# Download Kaggle data using Kaggle API
#!kaggle competitions download -c dat300-ca1-autumn-2019


In [ ]:
# Unzip Kaggle data
#!unzip "dat300-ca1-autumn-2019.zip" -d "/tmp/whatever"

In [2]:
#choose here which imputation methods to use:

all_imputation_methods = ['col_means', 'correlated_columns']
imputation_methods = ['correlated_columns', 'col_means']



In [3]:
# List files in directory

if os.name == 'nt':
    workdir = 'C://Users//thomoe//Documents//myDAT300//dat300-ca1-autumn-2019//'
elif os.name == 'posix':
    workdir = '/Users/jorgenkongsro/Downloads/dat300-ca1-autumn-2019/'
    
os.listdir(workdir)


['features_correlations.txt',
 'features_correlation_matrix.txt',
 'features_correlation_matrix.xlsx',
 'X_test.csv',
 'X_train.csv',
 'y_test_sampleSubmission.csv',
 'y_train.csv']

In [4]:
def import_data(csv_file):
    """ Import data from csv file

    :param data: a .csv separated dataset
    :return: a pandas data array, df
    
    """
    
    
    df = dd.read_csv(csv_file)
    return df

x_train_df = import_data(workdir + 'X_train.csv')
x_test_df = import_data(workdir + 'X_test.csv')
y_train_df = import_data(workdir + 'y_train.csv')



In [5]:
#get dimensions etc:

x_train_df_nrows = len(x_train_df)
x_train_df_ncols = len(x_train_df.columns)
x_test_df_nrows = len(x_test_df)
x_test_df_ncols = len(x_test_df.columns)

x_train_df_colnames = list(x_train_df.columns)



In [6]:
def percent_missing(dataframe):
    """ Check for percent missing values in dataframe
    :param data: dataframe
    :return: dataframe
    
    """
    missing_values = dataframe.isnull().sum()
    
    with ProgressBar():
        percent_missing = ((missing_values / dataframe.index.size) * 100).compute()
        
    return percent_missing


print(percent_missing(x_train_df))

"""
note: the results indicate that the features come in "tripets", e.g. f1 to f3 have quite similar missing%. 
We could impute some values very precisely by insert the mean of the other values within the triplet, 
if only one or two values are missing
"""

[########################################] | 100% Completed | 39.1s
f1      3.628002
f2      3.754723
f3      3.836795
f4      4.133142
f5      4.299061
f6      4.377457
f7      9.171993
f8      9.276405
f9      9.410288
f10     3.628002
f11     3.754723
f12     3.836795
f13     4.133142
f14     4.299061
f15     4.377457
f16     9.171993
f17     9.276405
f18     9.410288
f19     3.628002
f20     3.754723
f21     3.836795
f22     4.133142
f23     4.299061
f24     4.377457
f25     9.171993
f26     9.276405
f27     9.410288
f28     3.628002
f29     3.754723
f30     3.836795
          ...   
f133    9.171993
f134    9.276405
f135    9.410288
f136    3.628002
f137    3.754723
f138    3.836795
f139    4.133142
f140    4.299061
f141    4.377457
f142    9.171993
f143    9.276405
f144    9.410288
f145    3.628002
f146    3.754723
f147    3.836795
f148    4.133142
f149    4.299061
f150    4.377457
f151    9.171993
f152    9.276405
f153    9.410288
f154    3.628002
f155    3.754723
f156    3.8367

'\nnote: the results indicate that the features come in "tripets", e.g. f1 to f3 have quite similar missing%. \nWe could impute some values very precisely by insert the mean of the other values within the triplet, \nif only one or two values are missing\n'

In [15]:
def nan_filter(dataframe):
    """ Filter out NaN
    :param data: dataframe
    :return: dataframe
    
    """
    filtered = dataframe.dropna(how='all', axis= 0, subset=None, thresh=None).compute()

    return filtered

filtered = nan_filter(x_train_df)
percent_missing(filtered)

#THIS ONE DOES NOT WORK, THE AXIS PARAMETER SEEMS TO HAVE BEEN DEPRECATED. 
#DROPPING A WITHIN SPECIFYING ROWS OR COLUMNS DID NOT REMOVE ANY OF EITHER

#remove columns where all observations are NaN: 
#print(len(x_train_df.columns))
#print(len(x_train_df))
#x_train_df_nafiltered = x_train_df.dropna(how='all').compute()
#print(len(x_train_df.columns))
#print(len(x_train_df))



TypeError: dropna() got an unexpected keyword argument 'axis'

In [7]:
def drop_columns(df, thresh_perc, ncols = None, nrows = None):
    """ Drop columns which have more than 50% missing
    :param data: dataframe
    :return: dataframe
    """
    
    if ncols == None: ncols = len(df.columns)
    if nrows == None: nrows = len(df)
    
    nmissing = list(df.isna().sum().compute())
    colstodrop = [a for a in range(ncols) if nmissing[a]/nrows*100 >= thresh_perc]
    colstodrop = [df.columns[a] for a in colstodrop]
    return df.drop(colstodrop, axis='columns')
   


In [8]:
#delete columns (features) with more than N % missing observations from training data set: 
missing_perc = 50
x_train_df = drop_columns(x_train_df, missing_perc, x_train_df_ncols, x_train_df_nrows)

diff = x_train_df_ncols - len(x_train_df.columns)
if diff > 0: 
    print('Deleted', diff, 'features because they had more than', missing_perc, 'percent missing observations')
    x_train_df_ncols = len(x_train_df.columns)
else:
    print('No features had more than', missing_perc, 'percent missing observations')    

No features had more than 50 percent missing observations


In [166]:
#look at distribution of missing values in rows, using a histogram: 
if dir().count('nacounts_rows') == 0:
    nacounts_rows = x_train_df.isna().sum(axis=1).compute()
nacounts_rows.plot.hist(bins=20, grid=True)

#based on the histogram, or whatever, select a threshold for dropping rows:
nacount_thresh = 150

#make boolean list where True means more NaN's than nacount_thresh:
rows_to_drop = list(nacounts_rows > nacount_thresh)

#transform this to integer list containing row numbers of rows to drop (should have found more pandas-like way..)
rows_to_drop = [a for a in range(len(to_drop_bool)) if to_drop_bool[a] == True]

#drop rows (WHY WON'T IT ACCEPT ROWS AS AXIS??):
x_train_df_rowdrop = x_train_df.drop(rows_to_drop, axis='index')


<IPython.core.display.Javascript object>

NotImplementedError: Drop currently only works for axis=1

In [167]:
#impute by inserting value from most closely correlated column: 

"""
I can't understand why this one doesn't work. The idea was to fill in NaN's from correlated columns.
The problem might be in the last line

"""

#will only correct if the correlation between columns is above this threshold: 
lowest_allowed_corr = 0.8

if 'correlated_columns' in imputation_methods:
    
    #if correlation matrix does not exist, first try to read it from file, if that does not work calculate it: 
    if dir().count('corrs') == 0: 
        try: 
            corrs = pd.read_csv(workdir + 'features_correlation_matrix.txt')
        except:
            corrs = x_train_df.corr('pearson')
    
    #impute for each feature feat:
    for feat in x_train_df_colnames:
        
        #order the feature names according to (absolute value of) correlations to feat:
        abscorr = [abs(a) for a in list(corrs[feat])]
        order = np.argsort(abscorr)[::-1]
        topfeatures = [x_train_df_colnames[a] for a in order]
        
        #remove features which are not sufficiently correlated to feat:
        mapper = dict(zip(x_train_df_colnames, abscorr))
        topfeatures = [a for a in topfeatures if mapper[a] >= lowest_allowed_corr]
        
        #correct using each feature in topfeatures, starting with the most strongly correlated otherfeature:
        for otherfeat in [a for a in topfeatures if a != feat]:
            x_train_df[feat] = x_train_df[feat].fillna(x_train_df[otherfeat])
    x_train_df.to_csv(workdir + 'X_train_corrimp80.csv')
    pm = percent_missing(x_train_df)
    print(pm)
    
    



MemoryError: 

In [170]:
dir()


['In',
 'Out',
 'ProgressBar',
 '_',
 '_106',
 '_107',
 '_108',
 '_11',
 '_114',
 '_115',
 '_124',
 '_132',
 '_133',
 '_135',
 '_136',
 '_137',
 '_138',
 '_14',
 '_141',
 '_143',
 '_144',
 '_149',
 '_152',
 '_153',
 '_154',
 '_155',
 '_157',
 '_158',
 '_159',
 '_160',
 '_161',
 '_162',
 '_163',
 '_164',
 '_165',
 '_168',
 '_3',
 '_30',
 '_34',
 '_45',
 '_54',
 '_55',
 '_58',
 '_64',
 '_65',
 '_97',
 '__',
 '___',
 '__builtin__',
 '__builtins__',
 '__doc__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '_dh',
 '_i',
 '_i1',
 '_i10',
 '_i100',
 '_i101',
 '_i102',
 '_i103',
 '_i104',
 '_i105',
 '_i106',
 '_i107',
 '_i108',
 '_i109',
 '_i11',
 '_i110',
 '_i111',
 '_i112',
 '_i113',
 '_i114',
 '_i115',
 '_i116',
 '_i117',
 '_i118',
 '_i119',
 '_i12',
 '_i120',
 '_i121',
 '_i122',
 '_i123',
 '_i124',
 '_i125',
 '_i126',
 '_i127',
 '_i128',
 '_i129',
 '_i13',
 '_i130',
 '_i131',
 '_i132',
 '_i133',
 '_i134',
 '_i135',
 '_i136',
 '_i137',
 '_i138',
 '_i139',
 '_i14',
 '_i140',
 '_

In [169]:
del x_train_df

In [129]:
pm = percent_missing(x_train_df_corrimp)
print(pm)

[########################################] | 100% Completed |  1.4s
f1      3.628002
f2      3.754723
f3      3.754723
f4      4.133142
f5      4.133142
f6      4.133142
f7      9.171993
f8      9.276405
f9      9.410288
f10     3.628002
f11     3.754723
f12     3.836795
f13     4.133142
f14     4.299061
f15     4.377457
f16     9.171993
f17     9.276405
f18     9.410288
f19     3.628002
f20     3.754723
f21     3.836795
f22     4.133142
f23     4.299061
f24     4.377457
f25     9.171993
f26     9.276405
f27     9.410288
f28     3.628002
f29     3.754723
f30     3.836795
          ...   
f133    9.171993
f134    9.276405
f135    9.410288
f136    3.628002
f137    3.754723
f138    3.836795
f139    4.133142
f140    4.299061
f141    4.377457
f142    9.171993
f143    9.276405
f144    9.410288
f145    3.628002
f146    3.754723
f147    3.836795
f148    4.133142
f149    4.299061
f150    4.377457
f151    9.171993
f152    9.276405
f153    9.410288
f154    3.628002
f155    3.754723
f156    3.8367

In [126]:
del x_train_df_corrimp

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,...,f153,f154,f155,f156,f157,f158,f159,f160,f161,f162
0,0.650080,0.307440,0.306270,0.186580,0.179110,0.178230,0.174430,0.151130,0.139410,1.837900,...,0.130790,0.119300,0.119300,0.119300,0.119300,0.119300,0.119300,0.119300,0.119300,0.119300
1,0.098689,0.090486,0.063531,0.050347,0.049322,0.039360,0.007132,-0.016892,-0.022459,-0.310620,...,-0.038232,0.040767,0.040767,0.040767,-0.003073,-0.003073,-0.003073,-0.003073,-0.003073,-0.003073
2,0.079645,0.074518,0.071149,0.052691,0.039653,0.039214,0.037016,0.029252,0.010062,-0.348400,...,-0.038232,0.040767,0.040767,0.040767,0.040767,0.040767,0.040767,0.040767,0.040767,0.040767
3,0.148200,0.101180,0.091218,0.090486,0.051666,0.038042,0.021781,0.006546,-0.008835,-0.286620,...,0.066729,0.051966,0.051966,0.051966,0.051966,0.051966,0.051966,0.051966,0.051966,0.051966
4,0.148200,0.101180,0.091218,0.089167,0.051666,0.043755,0.040093,0.021781,0.006546,-0.380270,...,0.066729,0.051966,0.051966,0.051966,0.051966,0.051966,0.051966,0.051966,0.051966,0.051966
5,0.125640,0.117730,0.093123,0.085359,0.081110,0.046831,NaN,NaN,NaN,-0.414100,...,NaN,0.044208,0.044208,0.044208,0.044208,0.044208,0.044208,NaN,NaN,NaN
6,0.048150,0.022660,0.016801,0.013871,0.013431,-0.001364,NaN,NaN,NaN,-0.443830,...,NaN,0.044208,0.044208,0.044208,0.044208,0.044208,0.044208,NaN,NaN,NaN
7,0.149230,0.040532,0.031743,0.008744,-0.010593,-0.087940,NaN,NaN,NaN,-0.087262,...,NaN,-0.009096,-0.009096,-0.009096,-0.009096,-0.009096,-0.009096,NaN,NaN,NaN
8,0.228480,0.209440,0.149230,0.142200,0.098250,0.040532,NaN,NaN,NaN,-0.087262,...,NaN,-0.009096,-0.009096,-0.009096,-0.009096,-0.009096,-0.009096,NaN,NaN,NaN
9,0.089167,0.073493,0.069537,0.055621,0.053277,0.043755,NaN,NaN,NaN,-0.492080,...,NaN,0.103310,0.103310,0.103310,0.103310,0.051966,0.051966,NaN,NaN,NaN


In [81]:
#impute by inserting the mean of the column in question, for all columns: 
 
if 'col_means' in imputation_methods:
    
    #calculate mean (note that axis needs to be 0 to get columns, which is weird)
    miin = x_train_df.mean(axis = 0).compute()

    x_train_df_imean = x_train_df.fillna(dict(miin))


In [ ]:
#impute by inserting the mean of the column in question, for all columns: 
 
if 'col_means' in imputation_methods:
    
    #calculate mean (note that axis needs to be 0 to get columns, which is weird)
    miin = x_test_df.mean(axis = 0).compute()

    x_test_df_imean = x_train_df.fillna(dict(miin))

Int64Index([    0,     1,     2,     3,     4,     5,     6,     7,     8,
                9,
            ...
            35898, 35899, 35900, 35901, 35902, 35903, 35904, 35905, 35906,
            35907],
           dtype='int64', length=3155759)

In [63]:
#use correlation between columns in order to impute: 

corr_thresh = 0.90

#for j in range(len(x_train_ncols)):
#    col_j = 
#    for k in range(len(x_train_ncols)) if j != k: 
 
#len(x_train_df.columns)

162

In [ ]:
def drop_rows(dataframe):
    """ Drop rows if more than 0 and less than 5% missing 
    :param data: dataframe
    :return: dataframe
    
    """
    rows_to_drop = list(percent_missing[(percent_missing > 0) & (percent_missing < 5)].index)
    data_clean = dataframe.dropna(subset=rows_to_drop)

In [ ]:
def impute_rows(dataframe,feature):
    """ Impute rows if more than 5% and less than 50% missing
    :param data: dataframe, feature
    :return: dataframe
    
    """
    value = median[feature]
    df_impute[feature] = df[feature].fillna(value)
    return df_impute

In [ ]:
# Print the size of a feature
print (x_train_df.count().compute())


In [ ]:
# Build model and train

# Split data into train and validation

# JØRGEN KAN BEGYNNE BYGGE DETTE

from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from dask.diagnostics import ProgressBar

bern = BernoulliNB()
with ProgressBar():
    bern.fit(x_train_df, y_train_df)